In [1]:
import numpy as np
from itertools import cycle
import os

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import datetime as dt
import subprocess
from scipy.ndimage import binary_fill_holes
from importlib import reload
# os.chdir("/home/149/ab8992/cosima_regional/development/regional-mom6/")
from pathlib import Path
import regional_mom6 as rm
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 8,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45439,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:46223,Total threads: 2
Dashboard: /proxy/38813/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:41861,


## Convert restart to initial condition. No other changes

In [7]:
res = "10"

mask = xr.open_mfdataset(f"/g/data/nm03/ab8992/ttide-inputs/full-{res}/ocean_mask.nc").mask
restart = xr.open_mfdataset(f"/g/data/nm03/ab8992/ttide-inputs/DE-restarts/beamless-{res}/MOM.res*.nc",decode_cf = False,decode_times=False)

restart = restart[["lath","lonh","Layer","Temp","Salt","u","lonq","v","sfc","latq"]]

mask = mask.where(mask>0)

restart_masked = xr.Dataset(
    data_vars = dict(
        Temp = (["Layer","yh","xh"],restart.Temp.rename({"lonh":"xh","lath":"yh"}).isel(Time = 0).values),
        Salt = (["Layer","yh","xh"],restart.Salt.rename({"lonh":"xh","lath":"yh"}).isel(Time = 0).values),
        u = (["Layer","yh","xq"],restart.u.rename({"lonq":"xq","lath":"yh"}).isel(Time = 0).values),
        v = (["Layer","yq","xh"],restart.v.rename({"lonh":"xh","latq":"yq"}).isel(Time = 0).values),
        sfc = (["yh","xh"],restart.sfc.rename({"lonh":"xh","lath":"yh"}).isel(Time = 0).values),
),
    coords = dict(
        Layer = restart.Layer.values,
        xh = restart.lonh.values,
        yh = restart.lath.values
    )
)

# Multiply each variable in restart by the mask
for var in ["Temp","Salt","sfc"]:
    restart_masked[var] = restart_masked[var]*mask.values

init_vels = xr.open_dataset(
    f"/g/data/nm03/ab8992/ttide-inputs/full-{res}/forcing/init_vel.nc"
    )

init_tracers = xr.open_dataset(
    f"/g/data/nm03/ab8992/ttide-inputs/full-{res}/forcing/init_tracers.nc"
    )

init_eta = xr.open_dataset(
    f"/g/data/nm03/ab8992/ttide-inputs/full-{res}/forcing/init_eta.nc"
    )
init_vels.u[:,:,:] = restart_masked.u[:,:,:].values
init_vels.v[:,:,:] = restart_masked.v[:,:,:].values
init_tracers.temp[:,:,:] = restart_masked.Temp[:,:,:].values
init_tracers.salt[:,:,:] = restart_masked.Salt[:,:,:].values
init_eta.eta_t[:,:] = restart_masked.sfc[:,:].values

init_tracers = init_tracers.ffill("xh").bfill("xh").ffill("yh").bfill("yh").ffill("zl")
init_eta = init_eta.ffill("xh").bfill("xh").ffill("yh").bfill("yh")

init_vels.to_netcdf(f"/g/data/nm03/ab8992/ttide-inputs/DE-inits/inits-{res}/forcing/init_vel.nc")
init_tracers.to_netcdf(f"/g/data/nm03/ab8992/ttide-inputs/DE-inits/inits-{res}/forcing/init_tracers.nc",mode = "w",encoding={
                "temp": {"_FillValue": -1e20, "missing_value": -1e20},
                "salt": {"_FillValue": -1e20, "missing_value": -1e20},
            })
init_eta.to_netcdf(f"/g/data/nm03/ab8992/ttide-inputs/DE-inits/inits-{res}/forcing/init_eta.nc")


